In [40]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import neologdn

from classification.tools.parser import MessageManager, CabochaParser
from classification.tools.loader import BookManager

In [79]:
bookmanager = BookManager()
manager = MessageManager(parser=CabochaParser())
urls = bookmanager.load('101')
text = ''
for i, url in enumerate(urls):
    if i>20:
        break
    with open(url, 'rt') as f:
        temp = f.read()
    data = manager.extract_message(temp)
    parsed = data.text.replace('\n', '')
    result = neologdn.normalize(parsed)
    text += result
    print(i)

0
1


2


3
4


5


6
7


...

12
13
14


15


16
17


18


19
20


In [81]:
# 文書生成
path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
#text = open(path).read().lower()
print('corpus length:', len(text))

chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))


corpus length: 100908
total chars: 1855


In [82]:
maxlen = 40
step = 3
sentences = [] # 40文字のセンテンス
next_chars = [] # 'あ'などの1文字
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sentences:', len(sentences))

nb sentences: 33623


In [83]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

'よ'

In [84]:
# Model
model = Sequential()
model.add(LSTM(128, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

In [85]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [88]:
# Train
for iteration in range(1, 10):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x, y, batch_size=128, epochs=1, verbose=False)
    
    



--------------------------------------------------
Iteration 1



--------------------------------------------------
Iteration 2



--------------------------------------------------
Iteration 3



--------------------------------------------------
Iteration 4



--------------------------------------------------
Iteration 5



--------------------------------------------------
Iteration 6



--------------------------------------------------
Iteration 7



--------------------------------------------------
Iteration 8



--------------------------------------------------
Iteration 9


In [89]:
start_index = random.randint(0, len(text) - maxlen - 1)
print(start_index)

for diversity in [1.0, 1.2, 1.5]:
    print()
    print('---- diversity:', diversity)
    
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)
    
    for i in range(200):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.
            
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]
        
        generated += next_char
        sentence = sentence[1:] + next_char
        
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()


57395

---- diversity: 1.0
----- Generating with seed: "風が発生し、その音が耳を駆け抜けて行きます。私はもう一度手を伸ばしました。今度は"
風が発生し、その音が耳を駆け抜けて行きます。私はもう一度手を伸ばしました。今度は地

メ

な

い

ん

...

る

私

。

大

き

In [97]:
sentence = text[start_index: start_index + maxlen]
for i in range(140):
    x_pred = np.zeros((1, maxlen, len(chars)))
    for t, char in enumerate(sentence):
        x_pred[0, t, char_indices[char]] = 1.
        
    preds = model.predict(x_pred, verbose=0)[0]
    next_index = sample(preds, 1.2)
    next_char = indices_char[next_index]
    
    generated += next_char
    sentence = sentence[1:] + next_char
    
    sys.stdout.write(next_char)
    sys.stdout.flush()

の

プ

ロ

ー

別

...

し

た

飛

ん

で